In [2]:
#Функция для удаления дубликатов
def unique(lst):
    seen = set()
    result = []
    for x in lst:
        if x in seen:
            continue
        seen.add(x)
        result.append(x)
    return result

In [3]:
#вывести список групп
import sqlite3
#Создание списка из всех направлений...
 
conn = sqlite3.connect("stud_spisok_ege.db")
cursor = conn.cursor()

#выбираем прошедшие данные чтобы их вывести...
sql = "SELECT grp_spec_grp_spec_name FROM stud_spisok "
cursor.execute(sql)
#print(cursor.fetchall())
specials=cursor.fetchall()

conn.commit()
conn.close()

#Делаем список из специальностей
spec=[]
for i in range(len(unique(specials))):
    spec.append(unique(specials)[i][0])
    print(spec[i])

Экономика
Бухгалтерский учет, анализ и аудит
Финансы и кредит
Коммерция
Менеджмент
Экономическая теория
Национальная экономика
Статистика
Управление персоналом
Мировая экономика
Экономика труда
Экономика (беспрофильный бакалавриат)
Общеэкономическая подготовка
Экономическая безопасность


In [3]:
#Возьмем оценки за 1 семестр у Экономики по математическому анализу

In [4]:
#Ищем id студентов выбранной специальности
import sqlite3
 
conn = sqlite3.connect("stud_spisok_ege.db")
cursor = conn.cursor()

#выбираем прошедшие данные чтобы их вывести...
sql = "SELECT stud_kod FROM stud_spisok WHERE grp_spec_grp_spec_name = ?"
cursor.execute(sql,[(spec[0])])
#print(cursor.fetchall())
students_kod=cursor.fetchall()

conn.commit()
conn.close()

kod=[]
for i in range(len(students_kod)):
    kod.append(students_kod[i][0])
#print(kod)

In [5]:
slov_kods=dict.fromkeys(kod)

In [6]:
#Находим список предметов, по которым были экзамены у выбранного направления
def predms(ID,semestr):
    import sqlite3
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT predmet_name FROM stud_spisok_ball WHERE stud_kod = ? AND cd_type_name= ? AND semestr= ?"
    cursor.execute(sql,[(ID),(u'\u042d\u043a\u0437\u0430\u043c\u0435\u043d'),(semestr)])
    #print(cursor.fetchall())
    students_kod=cursor.fetchall()

    conn.commit()
    conn.close()

    predm=[]
    for i in range(len(unique(students_kod))):
        predm.append(unique(students_kod)[i][0])
    return(predm)

In [65]:
#predms(kod[0])
#Это матанализ:
#u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0438\u0439 \u0430\u043d\u0430\u043b\u0438\u0437'

In [7]:
for i in predms(kod[0],1):
    print(i)

Философия
Математический анализ
Линейная алгебра
Психология


In [8]:
#Выводим оценки за выбранный предмет для каждого id:
#Ищем id студентов выбранной специальности
def markses(ID, predmet,semak):
    import sqlite3
 
    
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()
    
    #matan='Математический анализ'

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT ball FROM stud_spisok_ball WHERE stud_kod = ? AND predmet_name=? AND semestr=?"
    cursor.execute(sql,[(ID),(predmet),(semak)])
    #print(cursor.fetchall())
    marks=cursor.fetchall()
    
    
    sql_11 = "SELECT ball FROM stud_spisok_ball WHERE predmet_name=?"
    cursor.execute(sql_11,[(predmet)])
    maximum=cursor.fetchall()
    
    conn.commit()
    conn.close()
    
    new_11=0
    for i in maximum:
        if i[0]==None: continue
        if new_11<i[0]: new_11=i[0]

    new=[]
    for i in marks:
        if i[0]==None: new.append(0)
        else:new.append(i[0]*100/new_11)
    return(new)    

In [9]:
markses(kod[0],'Теория вероятностей и математическая статистика',3)

[47.5, 60.0, 52.5]

In [10]:
crot=[100.0,100.0,100.0]

In [52]:
#Делаем словарь из баллов и id студента
for i in kod:
    slov_kods[i]=markses(i, 'Линейная алгебра',1)

NameError: name 'slov_kods' is not defined

In [79]:
#slov_kods

In [11]:
#Эвклидово расстояние
def EuclideanDistance(list1, list2):
    
    #Если списки не равны, то уравниваем их:
    if len(list1)!=len(list2):
        if len(list1)>len(list2):
            list2+=[0]*abs(len(list1)-len(list2))
        else: 
            list1+=[0]*abs(len(list1)-len(list2))
                           
    Eukl=0
    for i in range(len(list1)):
        Eukl+=(list1[i]-list2[i])**2
    return(Eukl**(1/2))

In [12]:
EuclideanDistance(markses(kod[0], 'Линейная алгебра',1), markses(kod[1], 'Линейная алгебра',1))

32.01562118716424

In [13]:
#Поиск k- ближайших соседей по Евклидому расстоянию
# Указываем на соседей только один раз!!!
def SearchNeighbours(point,Neibhs,k):
    #Ищем расстояния
    Dists=[]
    for Neb in Neibhs:
        Dists.append((Neb,EuclideanDistance(markses(point, 'Линейная алгебра',1), markses(Neb, 'Линейная алгебра',1))))
        # Выбираем к- ближайших соседей
        Dists.sort(key=lambda i: i[1])
    return(Dists[:k])    

In [14]:
#Поиск k- ближайших соседей по Евклидому расстоянию
# Указываем на соседей только один раз!!!
def AllNeighbours(point,Neibhs):
    #Ищем расстояния
    Dists=[]
    for Neb in Neibhs:
        Dists.append((Neb,EuclideanDistance(markses(point, 'Линейная алгебра',1), markses(Neb, 'Линейная алгебра',1))))
        # Выбираем к- ближайших соседей
        Dists.sort(key=lambda i: i[1])
    return(Dists) 

In [15]:
sosedy=AllNeighbours(kod[0],kod[1:])

In [295]:
#Поиск с "кротом"
#Поиск k- ближайших соседей по Евклидому расстоянию
# Указываем на соседей только один раз!!!
def SearchNeighbours(Neibhs,k):
    #Ищем расстояния
    Dists=[]
    for Neb in Neibhs:
        Dists.append((Neb,EuclideanDistance(crot, markses(Neb, 'Математический анализ',1))))
        # Выбираем к- ближайших соседей
        Dists.sort(key=lambda i: i[1])
    #return(Dists[:k])

In [296]:
SearchNeighbours(kod,7)

[(63113, 31.7214438511238),
 (57051, 32.69174207655505),
 (50481, 32.977264895682296),
 (50636, 35.35533905932738),
 (302497, 35.35533905932738),
 (312601, 35.35533905932738),
 (489666, 35.35533905932738)]

In [16]:
SerNeb=[(63113, 31.7214438511238),
 (57051, 32.69174207655505),
 (50481, 32.977264895682296),
 (50636, 35.35533905932738),
 (302497, 35.35533905932738),
 (312601, 35.35533905932738),
 (489666, 35.35533905932738)]

In [17]:
#Мы нашли k- ближайших соседей
SerNeb=SearchNeighbours(kod[0],kod,7)

In [18]:
SerNeb

[(50095, 0.0),
 (57028, 2.5),
 (58591, 2.5),
 (64500, 2.5),
 (64547, 3.5355339059327378),
 (65457, 3.5355339059327378),
 (63198, 4.330127018922194)]

In [19]:
# Выводим список среднеарифметических оценок для каждого из ближайших выбранных соседей
import numpy as np
def AverageMarks(spp, predmets,semestrs):
    new=[]
    schet=0
    for t in range(len(spp)):
        if t==0: 
            new=np.array(markses(spp[t][0], predmets,semestrs))
            zh=len(new)
        else: 
            if markses(spp[t][0], predmets,semestrs)==[]: schet+=1
            else:
                if len(np.array(markses(spp[t][0], predmets,semestrs)))<zh: 
                    new+=np.array(markses(spp[t][0], predmets,semestrs)+[0]*(zh-len(np.array(markses(spp[t][0], predmets,semestrs)))))
                else:    
                    new+=np.array(markses(spp[t][0], predmets,semestrs)[:zh])
    new1=list(new)
    for i in range(len(new1)): 
        new1[i]/=(len(spp)-schet)
        #new1[i]/=len(new1)
    return(new1)    

In [20]:
AverageMarks(SerNeb,'Теория вероятностей и математическая статистика',3)

[53.92857142857143, 49.285714285714285, 60.0]

In [21]:
#Предметы,похожие на выбранный нами.
SamePredmets=['Математический анализ',
 'Математический анализ',
 'Теория вероятностей и математическая статистика',
  'Статистика',
 'Финансы',
 'Мировая экономика и международные экономические отношения',
 'Маркетинг',
 'Международные инвестиции']

In [22]:
#Получаем весь прогноз по человеку
FinishAverage=[]
for i in range(len(SamePredmets)):
    #ListForStudent.append( markses(kod[0],SamePredmets[i],i+1)) #???
    FinishAverage+=AverageMarks(SerNeb,SamePredmets[i],i+1)
print(FinishAverage)    

[56.785714285714285, 67.5, 45.714285714285715, 63.57142857142857, 51.07142857142857, 43.214285714285715, 53.92857142857143, 49.285714285714285, 60.0, 54.285714285714285, 19.642857142857142, 19.821428571428573, 15.357142857142858, 40.0, 16.9811320754717, 19.40700808625337, 18.86792452830189, 8.355795148247978, 53.099730458221025, 22.857142857142858, 13.200000000000001, 23.714285714285715, 22.571428571428573, 58.0, 36.42857142857143, 41.42857142857143, 24.285714285714285, 70.0, 35.714285714285715, 50.0, 38.57142857142857, 68.57142857142857]


In [25]:
#Проверка результатов на самом тестируемомо человеке.
def testingOnPerson(person):
    proverkaNaCheloveke=[]
    for i in range(len(SamePredmets)):
        proverkaNaCheloveke+=markses(person,SamePredmets[i],i+1)
    return(proverkaNaCheloveke)    

In [26]:
RaitForKod=testingOnPerson(kod[0])

In [28]:
#Лучше не запускать!))
#Ищем оптимальное число соседей для максимально точных результатов:
ListOfSumm=[]
SummOtkl=0
ForGradient=[]
for K in range(1,10):
    SummOtkl=0
    #dannK=SearchNeighbours(kod[0],kod[1:],1)
    #Получаем весь прогноз по человеку
    FinishAverage=[]
    for i in range(len(SamePredmets)):
        #ListForStudent.append( markses(kod[0],SamePredmets[i],i+1)) #???
        FinishAverage+=AverageMarks(sosedy[:K],SamePredmets[i],i+1)
    #Теперь ищем отклонение:
    for i in range(min(len(FinishAverage),len(RaitForKod))):
        SummOtkl+=abs(FinishAverage[i]-RaitForKod[i])
    #Теперь надо получить среднеарифметическое из данных отклонений
    ListOfSumm.append(SummOtkl/min(len(FinishAverage),len(RaitForKod)))
print(ListOfSumm)                      

[6.865439690026954, 8.351583557951482, 7.330160601976639, 7.978536682951483, 7.103110680592992, 6.907691065813116, 6.37962853773585, 6.7019841854784366, 6.810491129922881]


In [22]:
#Вывод наилучших и наихулших сценариев
def schenaries(naighbours):
    bestAverage=[]
    worstAverage=[100000]
    for t in list(np.array(naighbours)[:,0]):
        if sum(testingOnPerson(t))> sum(bestAverage):
            bestAverage=testingOnPerson(t)
        if sum(testingOnPerson(t))< sum(worstAverage):
            worstAverage=testingOnPerson(t)
    return(bestAverage, worstAverage)        

In [18]:
#schenaries(SearchNeighbours(kod[0],kod[1:],6))

In [133]:
FinishAverage=[]
for i in range(len(SamePredmets)):
    #ListForStudent.append( markses(kod[0],SamePredmets[i],i+1)) #???
    FinishAverage+=AverageMarks(sosedy[:i+1],SamePredmets[i],i+1)

In [13]:
#Выводим общую прямую по предметам    

In [278]:
# Выводим список среднеарифметических оценок для каждого из ближайших выбранных соседей
import numpy as np
def AverageGr(spp, predmets,semestrs):
    new=[]
    schet=0
    for t in range(len(spp)):
        if t==0: 
            new=np.array(markses(spp[t], predmets,semestrs))
            zh=len(new)
        else: 
            if markses(spp[t], predmets,semestrs)==[]: schet+=1
            else:
                if len(np.array(markses(spp[t], predmets,semestrs)))<zh: 
                    new+=np.array(markses(spp[t], predmets,semestrs)+[0]*(zh-len(np.array(markses(spp[t], predmets,semestrs)))))
                else:    
                    new+=np.array(markses(spp[t], predmets,semestrs)[:zh])
    new1=list(new)
    for i in range(len(new1)): 
        new1[i]/=(len(spp)-schet)
        #new1[i]/=len(new1)
    return(new1)    

In [20]:
#Получаем весь прогноз по человеку
kodv=kod
FinishAverage=[]
for i in range(len(SamePredmets)):
    #ListForStudent.append( markses(kod[0],SamePredmets[i],i+1)) #???
    FinishAverage+=AverageGr(kodv,SamePredmets[i],i+1)
print(FinishAverage)  

[59.721059972106, 57.76150627615063, 58.59832635983263, 60.01326579835987, 57.76290400385914, 58.61070911722142, 57.61547911547912, 56.518427518427515, 60.097051597051596, 48.09099321406539, 27.346391116594695, 23.395280690931525, 34.28701418877236, 44.451727328809376, 23.935851432916465, 24.063340620692717, 22.16744688940628, 12.78798536191246, 54.36491849246708, 25.477396021699818, 15.711754068716099, 26.06618444846293, 31.259493670886076, 63.047197106690774, 38.915821033210335, 51.40659594095941, 28.81872693726937, 64.31988007380073, 40.49886621315192, 44.30839002267573, 50.385487528344676, 73.51473922902494]


In [35]:
#Строим подпространство для дальнейшего анализа данных
Underspace=np.array(SearchNeighbours(kod[0],kod[1:],500))[:,0]

In [60]:
Underspace=list(Underspace)
Und=Underspace[:200]

In [61]:
#Мы нашли k- ближайших соседей
SerNebUnder=SearchNeighbours(kod[0],Und,10)

In [62]:
SerNebUnder

[(57028.0, 2.5),
 (58591.0, 2.5),
 (64500.0, 2.5),
 (64547.0, 3.5355339059327378),
 (65457.0, 3.5355339059327378),
 (63198.0, 4.330127018922194),
 (51049.0, 5.0),
 (61291.0, 5.0),
 (50315.0, 5.5901699437494745),
 (50787.0, 5.5901699437494745)]

In [1]:
#Градиентные спуски для каждого из соседей
#Для построение трехмерных графиков
sch=-1
for rassmNeighbour in Underspace[:2]:
    sch+=1
    SummOtkl=0
    ForGradient=[]
    for K in range(1,10):
        SummOtkl=0
        #Получаем весь прогноз по человеку
        FinishAverage=[]
        for i in range(len(SamePredmets)):
            #ListForStudent.append( markses(kod[0],SamePredmets[i],i+1)) #???
            FinishAverage+=AverageMarks(SearchNeighbours(rassmNeighbour,Und,10)[:K],SamePredmets[i],i+1)
        #Теперь ищем отклонение:
        for i in range(min(len(FinishAverage),len(RaitForKod))):
            SummOtkl+=abs(FinishAverage[i]-RaitForKod[i])
        #Теперь надо получить среднеарифметическое из данных отклонений
        ListOfSumm.append([sch,SummOtkl/min(len(FinishAverage),len(RaitForKod)),K-1])
print(ListOfSumm) 

NameError: name 'Underspace' is not defined

In [191]:
import numpy as np
Itog_list=np.array([[0, 6.865439690026954, 0],
 [0, 8.127585916442047, 1],
 [0, 5.851791329739442, 2],
 [0, 6.450027375336927, 3],
 [0, 5.841810983827492, 4],
 [0, 5.791875842318059, 5],
 [0, 5.929524651841869, 6],
 [0, 6.168390961927224, 7],
 [0, 6.460990261867326, 8],
 [1, 11.000336927223717, 0],
 [1, 7.900101078167117, 1],
 [1, 6.642632524707996, 2],
 [1, 6.186320754716981, 3],
 [1, 6.206502695417789, 4],
 [1, 6.654818059299191, 5],
 [1, 6.855843280708511, 6],
 [1, 6.646527242972661, 7],
 [1, 7.069079439952083, 8],
 [2, 5.000336927223717, 0],
 [2, 9.900101078167117, 1],
 [2, 6.642632524707996, 2],
 [2, 6.186320754716981, 3],
 [2, 6.206502695417789, 4],
 [2, 6.654818059299191, 5],
 [2, 6.855843280708511, 6],
 [2, 6.646527242972661, 7],
 [2, 7.069079439952083, 8],
 [3, 12.000336927223717, 0],
 [3, 9.935101078167117, 1],
 [3, 6.642632524707996, 2],
 [3, 5.086320754716981, 3],
 [3, 6.103502695417789, 4],
 [3, 6.654818059299191, 5],
 [3, 6.745843280708511, 6],
 [3, 6.686527242972661, 7],
 [3, 7.157079439952083, 8],
 [4, 12.000336927223717, 0],
 [4, 10.9101078167117, 1],
 [4, 8.642632524707996, 2],
 [4, 6.186320754716981, 3],
 [4, 5.806502695417789, 4],
 [4, 5.754818059299191, 5],
 [4, 6.055843280708511, 6],
 [4, 6.146527242972661, 7],
 [4, 7.009079439952083, 8],
 [5, 8.70336927223717, 0],
 [5, 9.62101078167117, 1],
 [5, 5.642632524707996, 2],
 [5, 6.186320754716981, 3],
 [5, 5.706502695417789, 4],
 [5, 4.995818059299191, 5],
 [5, 4.955843280708511, 6],
 [5, 5.146527242972661, 7],
 [5, 6.068079439952083, 8],
 [6, 4.200336927223717, 0],
 [6, 9.900101078167117, 1],
 [6, 6.222632524707996, 2],
 [6, 5.986320754716981, 3],
 [6, 6.003502695417789, 4],
 [6, 6.954818059299191, 5],
 [6, 6.745843280708511, 6],
 [6, 7.885127242972661, 7],
 [6, 7.157079439952083, 8],
 [7, 5.200336927223717, 0],
 [7, 10.120101078167117, 1],
 [7, 6.342632524707996, 2],
 [7, 5.276320754716981, 3],
 [7, 6.973502695417789, 4],
 [7, 6.454818059299191, 5],
 [7, 6.567843280708511, 6],
 [7, 6.765527242972661, 7],
 [7, 7.150079439952083, 8],
 [8, 11.099336927223717, 0],
 [8, 10.700101078167117, 1],
 [8, 8.312632524707996, 2],
 [8, 7.926320754716981, 3],
 [8, 7.806502695417789, 4],
 [8, 7.354818059299191, 5],
 [8, 7.145843280708511, 6],
 [8, 6.806527242972661, 7],
 [8, 7.069079439952083, 8],
 [9, 4.950336927223717, 0],
 [9, 9.890101078167117, 1],
 [9, 5.242632524707996, 2],
 [9, 5.586320754716981, 3],
 [9, 5.606502695417789, 4],
 [9, 5.254818059299191, 5],
 [9, 4.985843280708511, 6],
 [9, 5.156527242972661, 7],
 [9, 6.269079439952083, 8],
 [10, 1.210336627223717, 0],
 [10, 8.166101078167117, 1],
 [10, 7.222632524707996, 2],
 [10, 6.086320754716981, 3],
 [10, 6.103502695417789, 4],
 [10, 6.054818059299191, 5],
 [10, 6.755843280708511, 6],
 [10, 7.186527242972661, 7],
 [10, 7.157079439952083, 8]])

In [192]:
82.23084100727756/11

7.475531000661596

In [193]:
OptimumK

[7.475531000661596,
 7.475531000661596,
 7.475531000661596,
 7.475531000661596,
 7.475531000661596,
 7.475531000661596,
 7.475531000661596,
 7.475531000661596,
 7.475531000661596]

In [196]:
OptimumK=[]
sch=0
for k in range(9):
    vv=0
    vl=k
    for neighour in range(11):
        vv+=Itog_list[:,1][vl]
        vl+=9
        #print(Itog_list[:,1][k])
    sch+=1    
    OptimumK.append(vv/11)

In [197]:
OptimumK

[7.475531000661596,
 9.560955740014714,
 6.673465143347218,
 6.103930447500613,
 6.214985267091398,
 6.234641494119089,
 6.3272688599024525,
 6.47647849015035,
 6.8759804237625595]